In [2]:
#install
#@markdown To set up Simple Stable, click on the triangle play button on the left. The interface will appear below.
#@markdown
#@markdown To hide large or annoying code on the left, double-click on this text.
#@markdown
#@markdown To open the interface in a new tab or window, click on the link after "Running on public URL".\
#@markdown When loading a new model for the first time, it may take a few minutes to download, but switching between already downloaded models will be faster.\
#@markdown Please note that time estimates may not be accurate.

starting_model_to_load = "Stable Diffusion 1.5" #@param ["Stable Diffusion 1.4","Stable Diffusion 1.5","Stable Diffusion 2.0 (512)","Stable Diffusion 2.0 (768)","Stable Diffusion 2.1 (512)","Stable Diffusion 2.1 (768)","Dreamlike Diffusion","Openjourney","Openjourney V2 Beta","Oldjourney Lite","Oldjourney Ultra","seek.art_MEGA","Sygil Diffusion","Analog Diffusion","Dreamlike Photoreal","Anything v3.0","Anything v4.0","Waifu Diffusion 1.4 (Epoch 1 Test Version)","Plat Diffusion (Waifu 1.4 finetune)","AnimeChanStyle","Inkpunk Diffusion","Vivid Watercolors","Future Diffusion","Gigafractal Diffusion (768)","Van Gogh Diffusion","Corporate Memphis","Pokemon Diffusion","Pokemon Sugimori SD 2.1","Robo Diffusion","Classic Animation Diffusion","Modern Animation Diffusion","Ghibli Diffusion","Furry Diffusion"] {type:"string"}

download_a_collection_of_text_inversion_embeddings_and_load_them = False #@param{type: "boolean"}

test_mode = True
enable_attention_slicing = False
enable_xformers = False
%env PYTORCH_CUDA_ALLOC_CONF=garbage_collection_threshold:0.6,max_split_size_mb:128

try:
  import torch
  import src
  %cd /content/simplest-stable/
except ImportError as e:
  import os, subprocess
  %cd /content/
  subprocess.run(["git", "clone", "https://github.com/cadaeix/simplest-stable.git"])
  %cd /content/simplest-stable/
  if test_mode:
    subprocess.run(["git", "checkout", "test-branch"])
  subprocess.run(["pip3", "install", "-r", "requirements.txt"])
  if enable_xformers:
    subprocess.run(['pip', 'install', 'triton==2.0.0.dev20221202', 'xformers==0.0.16rc424'])
  subprocess.run(["pip3", "install", "gradio"])
  outputs_path = "/content/images/"
  subprocess.run(["mkdir", "-p", outputs_path])
  print(f"Outputs will be saved to {outputs_path}.")
  downloaded_embeds_path = None

import os, subprocess, sys, random, math, json, glob, logging
logging.disable(sys.maxsize)

# Pillow occasionally has odd import errors, the below fix them
import PIL.Image
if not hasattr(PIL.Image, 'Resampling'):
  PIL.Image.Resampling = PIL.Image

import PIL._util
if not hasattr(PIL._util, 'is_path'):
  from pathlib import Path
  def is_path(f):
    return isinstance(f, (bytes, str, Path))
  PIL._util.is_path = is_path

import PIL.TiffTags
if not hasattr(PIL.TiffTags, 'IFD'):
  PIL.TiffTags.IFD = 13
if not hasattr(PIL.TiffTags, 'LONG8'):
  PIL.TiffTags.LONG8 = 16

from datetime import datetime
from src.loading import prepare_pipe
from src.process import process_and_generate
from src.utils import free_ram
from src.gradio_ui import main
from IPython.display import display, clear_output
sys.path.append(".")

downloaded_embeds_path = None
if download_a_collection_of_text_inversion_embeddings_and_load_them:
  downloaded_embeds_path = "/content/embeds/"
  if not os.path.exists(downloaded_embeds_path):
    subprocess.run(["mkdir", "-p", downloaded_embeds_path])
    with open('src/resources/embeddings_links.json') as embeddingfile:
      embed_dict = json.load(embeddingfile)
    for embed in embed_dict["embeddings"]:
      subprocess.run(["wget", "-O", os.path.join(downloaded_embeds_path, embed["filename"]), embed["download_url"]])

try:
  outputs_folder
except NameError as e:
  outputs_path = "images/"
  if not os.path.exists(outputs_path):
      os.mkdir(outputs_path)
  outputs_folder = os.path.join(outputs_path, datetime.now().strftime("%Y_%m_%d"))

try:
  custom_models_folder
  custom_models_path = custom_models_folder if os.path.exists(custom_models_folder) else None
except NameError as e:
  custom_models_path = None

try:
  custom_embeddings_folder
  embeddings_path = custom_embeddings_folder if os.path.exists(custom_embeddings_folder) else None
except NameError as e:
  embeddings_path = None

try: 
  custom_randomizer_folder
  randomizer_folder = custom_randomizer_folder if os.path.exists(custom_randomizer_folder) else None
except NameError as e:
  randomizer_folder = None
  
clear_output(wait=False)

try:
  main(
    starting_model_to_load = starting_model_to_load, 
    outputs_folder = outputs_folder, 
    custom_models_path = custom_models_path,
    embeddings_path = embeddings_path, 
    downloaded_embeddings = downloaded_embeds_path,
    custom_randomizer_folder = randomizer_folder,
    enable_attention_slicing = enable_attention_slicing,
    enable_xformers = enable_xformers)
except Exception as e:
  print("Sorry, for some reason the interface is refusing to start. Try running this cell again or going to Runtime -> Disconnect and Delete Runtime and then rerunning this cell, and then if that fails, take a screenshot or copy the error below and ask @Cadaeic about it!")
  raise e

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Loaded the following embeddings: v1_admiralexclipse v1_bad_prompt v1_boris_buttons v1_cad_style_100 v1_CharTurner v1_naval_bicorne v1_naval_portrait v1_rainbow_ocean v1_spectral_style
Did not load the following embeddings (probably because they're for a different version of Stable Diffusion): psh pt16 v2_ActionHelper v2_AnimeScreenCap v2_cad_rainbow v2_cad_style_2_400 v2_cad_style_400 v2_CandyPunk v2_CarHelper v2_ChemPunk v2_CinemaHelper v2_classipeint v2_CutAway v2_double_exposure v2_dreamink v2_holopastel v2_HyperFluid v2_HyperSmoke v2_InkPunk768 v2_joe87_vibe v2_Kipaki v2_knollingcase v2_laxpeint v2_naval_portrait_400 v2_nipainterly v2_painterly v2_painterly_portrait v2_smoose v2_SocialistRealism v2_tf86movie v2_tfboxart v2_ThisHonor v2_VikingPunk v2_VintageHelper v2_nidream v2_niscreen v2_wat v2_where v2_why v2_wrong

Saving images to images/2023_02_04.
Found custom models folder at B:\Files\AI stuff\Models\models!
Found custom embeddings folder at B:\Files\AI stuff\Embeddings\embe